In [ ]:
import json
import numpy as np
import pandas as pd
import pyproj
import plotly.graph_objects as go

import parking_triangulation_utils

In [ ]:
# load vehicle detections
car_triangulations_df = pd.read_csv(
    "/home/sf_afn/Downloads/univrses_data/parked_car_triangulations_2023_04_24.csv"
)

# Load the parking segments
with open(
    "/home/sf_afn//Downloads/univrses_data/permitted_parking_linestrings.json"
) as user_file:
    parking_json = json.load(user_file)


In [ ]:
car_triangulations_df

In [ ]:
# ----> 1. convert loaded df crs
car_triangulations_df = parking_triangulation_utils.convert_crs(car_triangulations_df)
detections = car_triangulations_df["xy_3011"][:]
# 2.
car_triangulations_df["nearby_parking_ids"] = car_triangulations_df.apply(
    lambda row: parking_triangulation_utils.parking_proximity(
        row["xy_3011"], parking_json, radius=20
    ),
    axis=1,
)


# for point in detections:
#     nearby_parking_ids = parking_triangulation_utils.parking_proximity(point,
#                                                                        parking_json,
#                                                                        radius= 20)


In [ ]:
car_triangulations_df.info()


In [ ]:
# ----> 3.
# maybe like plot all car detections

len(car_triangulations_df["nearby_parking_ids"][1])

In [ ]:
# check if a cell is an empty list
mask = car_triangulations_df["nearby_parking_ids"].apply(lambda x: len(x) != 0)

car_tri_filtered_df = car_triangulations_df.loc[mask]

In [ ]:
car_tri_filtered_df


In [ ]:
def get_projections(parking_json, car_tri_filtered_df):
    for each in car_tri_filtered_df["nearby_parking_ids"]:
        print(each)
    # parking_triangulation_utils.parking_line_at_index(parking_json,)

In [ ]:
car_tri_filtered_df[["xy_3011", "nearby_parking_ids"]]

In [ ]:
# from tqdm import tqdm

In [ ]:
locate_closest_parking_idx(car_triangulations_df,parking_json)

In [ ]:
for idx, row in car_tri_filtered_df.iterrows():
    tri_point = row["xy_3011"]
    parking_lines_idx_list = row["nearby_parking_ids"]
    # print(parking_lines_idx_list)
    if len(row["nearby_parking_ids"]) == 1:
        # print(idx,row['nearby_parking_ids'])
        car_tri_filtered_df.loc[idx, "closest_parking_id"] = row["nearby_parking_ids"][0]
        # break
        # print(idx,row['xy_3011'],row['closest_parking_line'])
    else:
        proj_dict = {}
        for one_parking_line_id in row["nearby_parking_ids"]:
            parking_line_coords = parking_triangulation_utils.parking_line_at_index(
                parking_json, one_parking_line_id
            )
            parking_line_p = [parking_line_coords[0], parking_line_coords[-1]]
            
            proj = parking_triangulation_utils.project_point_on_line_segment(
                tri_point, parking_line_p
            )
            # print(idx,parking_line_coords[0],parking_line_coords[-1])
            proj_dist = [np.linalg.norm(tri_point - proj)] #list

            proj_dict[one_parking_line_id] = proj_dist 
            # print('got proj',proj_dist)
        # print(proj_dict)
        car_tri_filtered_df.loc[idx, "closest_parking_id"] = min(proj_dict, key=proj_dict.get)
        car_tri_filtered_df.loc[idx, "all_close_parking_id"] = [proj_dict]
        # print(tri_point,proj_list)

        # print(one_parking_line)

In [ ]:

parking_json["features"]

In [ ]:
print(car_tri_filtered_df['all_close_parking_id'])

In [ ]:
car_tri_filtered_df[:10]

In [ ]:
#retrive geometry by parking line idx

#copy the df for safe keeping

car_tri_filtered_df_copy = car_tri_filtered_df.copy(deep= True)

In [ ]:
car_tri_filtered_df_copy.head()

In [ ]:
car_tri_filtered_df_copy['closest_parking_id']= car_tri_filtered_df_copy.apply(lambda r: int(r['closest_parking_id']),axis=1)

In [ ]:
# add closest parking id geometry

car_tri_filtered_df_copy['closest_parking_geom']= car_tri_filtered_df_copy.apply(lambda row: parking_triangulation_utils
                                                       .parking_line_at_index(
                                                           parking_json, row['closest_parking_id']), 
                                                            axis=1)

In [ ]:
car_tri_filtered_df_copy['closest_parking_geom'].iloc[0]

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
#permitted parking all in blue
fig.add_trace(
    go.Scatter(
        x=[
            p[0]
            for f in parking_json["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        y=[
            p[1]
            for f in parking_json["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        name="permitted_parking",
    )
)
traces = []
# closest parking in red
for idx, row in car_tri_filtered_df_copy.iterrows() :
    trace = fig.add_trace(go.Scatter(x=[r[0] for r in row['closest_parking_geom']],
                       y=[r[1] for r in row['closest_parking_geom']], 
                       name= f"closest_parking_{row['closest_parking_id']}", 
                       mode= 'lines', 
                       marker_color='red'
                        ))               
    #traces.append(trace)

# plot the traces
#fig = go.Figure(data=traces)

# triangualtion point in ...
fig.add_trace(
    go.Scatter(
        x=[
            x for x in car_tri_filtered_df_copy['x_3011']
            ],
        y=[
           y for y in car_tri_filtered_df_copy['y_3011'] 
            ],
        name="Car triangulations",
        mode="markers",
        marker_color='green',
        marker_size= 5
              
    )
)

fig.update_layout(
    autosize= False,
    width = 1200,
    height = 1000
    
)
fig.update_yaxes(
    scaleanchor="x")
    

fig.show()


In [ ]:
car_tri_filtered_df_copy

In [ ]:
# car_tri_filtered_df_copy.to_csv('car_tri.csv')

In [ ]:
type(car_tri_filtered_df_copy.iloc[1]['closest_parking_id'])

In [ ]:
parking_occ_df = pd.DataFrame()

In [ ]:
parking_occ_df[['closest_parking_id','first_ts', 'last_ts']] =car_tri_filtered_df[['closest_parking_id',
                                                                                'first_timestamp',
                                                                                'last_timestamp']]

In [ ]:
parking_occ_df['parking_count'] = car_tri_filtered_df_copy.groupby('closest_parking_id').count()['xy_3011'].values

In [ ]:
parking_occ_df.index

In [ ]:
parking_occ_df.reset_index(inplace=True)

In [ ]:
parking_occ_df['closest_parking_id'] = parking_occ_df.apply(lambda r: int(r['closest_parking_id']),axis=1)
parking_occ_df['closes_parking_geom'] = parking_occ_df.apply(lambda row: parking_triangulation_utils
                                                       .parking_line_at_index(
                                                           parking_json, row['closest_parking_id']), 
                                                            axis=1)

In [ ]:
parking_occ_df

In [ ]:
car_tri_filtered_df_copy.groupby('closest_parking_id').count()['xy_3011']

In [ ]:
parking_id_grp_df.count()

In [ ]:
parking_id_grp_df